# Importation

In [1]:
import pandas as pd
import ast
# Création du barplot
import seaborn as sns
import matplotlib.pyplot as plt
#WordCloud
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Set stop words for English
stop_words = set(stopwords.words('english'))

# Recupération des données

In [2]:

# Lecture du fichier CSV
ratingdf = pd.read_csv('Books_rating_t.csv')
datadf = pd.read_csv('books_data_t.csv')

# Fonction utilitaire

In [3]:
def unwindColumns(df,columnsToUnwind):
    resdf = df
    for col in columnsToUnwind:
        resdf[col] = resdf[col].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) and x.startswith('[') else x)
        resdf = resdf.explode(col)
    return resdf

In [4]:
# Remove stop words
def remove_stop_words(text):
    tokens = word_tokenize(text)  # Tokenize the text
    filtered_tokens = [word for word in tokens if word.lower() not in stop_words]
    return ' '.join(filtered_tokens)  # Join tokens back into a string

# Exploration des données

In [ ]:
ratingdf.head(2)

In [ ]:
datadf.head(2)

In [ ]:
GenreDF = unwindColumns(datadf,['Genre'])
print(GenreDF['Genre'].value_counts().to_csv("temp.csv"))

# Preparation des donnée

In [ ]:
print(ratingdf.isnull().sum())
ratingdf = ratingdf.dropna(subset=['Titre'])
print(ratingdf.isnull().sum())
print(datadf.isnull().sum())
datadf = datadf.dropna(subset=['Titre'])
print(datadf.isnull().sum())


In [ ]:
merged_df = pd.merge(ratingdf, datadf, on='Titre')
merged_df.shape
merged_df.head(2)

# Distrubution en fonction des genres

In [10]:
GenreDF = unwindColumns(datadf,['Genre'])
genre_counts = GenreDF['Genre'].value_counts()

# Transformer en DataFrame
genre_counts_df = genre_counts.reset_index()

# Renommer les colonnes pour une meilleure lisibilité
genre_counts_df.columns = ['Genre', 'Nombre de livres']

# Afficher le DataFrame
genre_counts_df.to_csv("distribution_livre.csv")

In [ ]:
N = 20  # Ajuster selon vos besoins
top_genres = GenreDF['Genre'].value_counts().nlargest(N)

plt.figure(figsize=(12, 6))
sns.barplot(x=top_genres.index, y=top_genres.values)
plt.title(f'Top {N} genres les plus fréquents')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()

# Word Cloud Mot les plus frequents

In [ ]:
# 2. Les mots les plus fréquents dans les revues ayant obtenues un score >5
# (afficher sous forme de WordCloud)


commentaire =ratingdf[ratingdf['revue/score'] ==5]['revue/texte'].values
cleaned_commentaire = [remove_stop_words(comment) for comment in commentaire]

In [ ]:
text = ' '.join(map(str, cleaned_commentaire))

In [ ]:
wordcloud = WordCloud(
    width=1600, 
    height=800,
    background_color='white',
    stopwords=set(stopwords.words('english')),
    max_words=100
).generate(text)
plt.figure(figsize=(20,10))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Word Cloud des Revues 5 étoiles', fontsize=20)
plt.show()

# Livres les plus commenté

In [ ]:
N = 20  # Ajuster selon vos besoins
top_genres = ratingdf['Titre'].value_counts().nlargest(N)

plt.figure(figsize=(12, 6))
sns.barplot(x=top_genres.index, y=top_genres.values)
plt.title(f'Top {N} genres les plus fréquents')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()

# Livres les mieux noté

In [ ]:
# Calculer la moyenne de chaque colonne pour chaque genre
average_by_title = ratingdf.groupby('Titre')['revue/score'].mean().nlargest(N)

average_by_title

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split

# Séparation des features (X) et de la target (y)
X = merged_df.drop(columns=['revue/score'])
y = merged_df['revue/score']
le = OrdinalEncoder()
for colonne in X.select_dtypes(include=['object']).columns:
    X[[colonne]] = le.fit_transform(X[[colonne]])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Aperçu des données encodées
print("Aperçu des données encodées :")
X.head()


In [ ]:
# 8. Existe t-il une corrélation entre l’éditeur et le commentaire (score)
# 9. Existe-t-il une corrélation entre les auteurs et le commentaire (score).

# Créer la matrice de corrélation
import seaborn as sns
import matplotlib.pyplot as plt

correlation_matrix = X_train.corr()

plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, 
            annot=True,        
            cmap='coolwarm',   
            center=0,         
            fmt='.2f')   


plt.tight_layout()
plt.show()

In [ ]:
merged_df.head(2)